<a href="https://colab.research.google.com/github/nlavee/BashingTimer/blob/master/Assignment/Assignment_1_CS246.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Assignment 1

Vu nguyen

lammanhvu@stanford.edu


### Setup

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 45kB/s 
     |████████████████████████████████| 204kB 44.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=a2095cb2b4b73d366009e7141584f1518967952aea10dfdd84615985afd921de
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [2]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.


In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [4]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-04-08 01:01:11--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.222.61.237, 52.20.200.43, 3.225.89.236, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.222.61.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.06M  65.1MB/s    in 0.2s    

2021-04-08 01:01:12 (65.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14746350/14746350]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


**Note**: Assuming that data files are uploaded and present already.

## 1. Spark

Data: *soc-LiveJournal1Adj.txt*

In [6]:
#@title Read and glance at first 5 rows
live_journal_raw = spark.read.text('soc-LiveJournal1Adj.txt')
live_journal_raw.take(5)

[Row(value='0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'),
 Row(value='1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592'),
 Row(value='2\t0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878'),
 Row(value='3\t0,12,41,55,1532,12636,13185,27552,38737'),
 Row(value='4\t0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822')]

In [7]:
# Dealing in int since it might be simpler?
def turn_to_int_list(friends):
  if len(friends) == 0:
    return []
  else:
    return [int(i) for i in friends.split(",")]

user_friends_map = live_journal_raw.rdd.map(lambda user: 
                                            (int(user.value.split("\t")[0]), 
                                             turn_to_int_list(user.value.split("\t")[1])))
user_friends_map.take(3)

[(0,
  [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94]),
 (1,
  [0,
   5,
   20,
   135,
   2409,
   8715,
   8932,
   10623,
   12347,
   12846,
   13840,
   13845,
   14005,
   20075,
   21556,
   22939,
   23520,
   28193,
   29724,
   29791,
   29826,
   30691,
   31232,
   31435,
   32317,
   32489,
   34394,
   35589,
   35605,
   35606,
   35613,
   35633,
   35648,
   35678,
   38737,
   43

In [8]:
def get_all_potential_friends(friends):
  friend_pairs = []
  for friend_1 in friends:
    for friend_2 in friends:
      if friend_1 != friend_2:
        if friend_1 < friend_2:
          friend_pairs.append((friend_1, friend_2))
        else:
          friend_pairs.append((friend_2, friend_1))
  return friend_pairs

def get_all_non_potential_friends(existing_1, existing_list):
  non_potential_friends_pair = []
  for friend in existing_list:
    non_potential_friends_pair.append((existing_1, friend))
  return non_potential_friends_pair

# Get all the potential friends (this could include people who are already friends)
potential_friends = user_friends_map.flatMap(lambda x: get_all_potential_friends(x[1]))
# Get all the people who are already friends.
non_potential_friends = user_friends_map.flatMap(lambda x: get_all_non_potential_friends(x[0], x[1]))
# Remove those who are already friends from the list of people who could be friends.
true_potential_friends = potential_friends.subtract(non_potential_friends)

true_potential_friends.take(2)

[(1864, 6892), (1864, 6892)]

In [9]:
mututal_occurence_count = true_potential_friends.map(lambda x: (x, 1))
mututal_occurence_count.take(3)

[((15, 47), 1), ((15, 47), 1), ((12347, 13845), 1)]

In [10]:
# Count the number of mututal occurence
friend_pairs = mututal_occurence_count.reduceByKey(lambda a,b: a + b).cache()

In [11]:
friend_pairs.take(9)

[((10009, 37153), 4),
 ((1064, 33346), 2),
 ((44833, 48525), 2),
 ((1918, 3436), 2),
 ((626, 934), 2),
 ((21760, 24568), 2),
 ((22330, 22332), 2),
 ((7086, 32858), 2),
 ((9472, 39098), 2)]

In [12]:
def get_top_recs(user_id):
  # Filter specifically for interested users.
  interested_user_pairs = friend_pairs.filter(lambda x: x[0][0] == user_id or x[0][1] == user_id)
  user_id_nominations = interested_user_pairs.collect()
  # Sort the list to get what we want.
  user_id_nominations.sort(key = lambda x: (x[1], -x[0][1]), reverse=True)
  return user_id_nominations[0:10]

# Sanity Check
user_id = 11
recs = get_top_recs(user_id)
print(recs)

[((11, 27552), 8), ((11, 7785), 6), ((11, 27573), 6), ((11, 27574), 6), ((11, 27589), 6), ((11, 27590), 6), ((11, 27600), 6), ((11, 27617), 6), ((11, 27620), 6), ((11, 27667), 6)]


In [17]:
user_ids = [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]

for user in user_ids:
  print("User: ", user)
  recs = get_top_recs(user)
  print_recs = ''
  for rec in recs:
    print_recs += (" " + (str(rec[0][1]) if rec[0][1] != user else str(rec[0][0])))
  print(user, "\t", print_recs)
  print("-----")


User:  924
924 	  439 2409 6995 11860 15416 43748 45881
-----
User:  8941
8941 	  8943 8944 8940
-----
User:  8942
8942 	  8939 8940 8943 8944
-----
User:  9019
9019 	  9022 317 9023
-----
User:  9020
9020 	  9021 9016 9017 9022 317 9023
-----
User:  9021
9021 	  9020 9017 9016 9022 317 9023
-----
User:  9022
9022 	  9020 9019 9021 9016 317 9017 9023
-----
User:  9990
9990 	  13134 13478 13877 34299 34485 34642 37941
-----
User:  9992
9992 	  9989 9987 35667 9991
-----
User:  9993
9993 	  9991 13134 13478 13877 34299 34485 34642 37941
-----
